### setup

In [1]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re
import ast

from PIL import Image
import base64
import sys

from IPython.display import clear_output

## plotting
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from collections import Counter

## svg rendering 
# import ast
# from svgpathtools import parse_path, wsvg, svg2paths

#### paths etc.


In [5]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
stimulus_dir = os.getcwd()
analysis_dir = os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','helpers') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis','helpers'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
        
import svg_rendering_helpers as srh 
reload(srh)

<module 'svg_rendering_helpers' from '/Users/judithfan/semantic_parts/analysis/helpers/svg_rendering_helpers.pyc'>

#### load in group data csv

In [46]:
path_to_group_data = os.path.join(csv_dir,'semantic_parts_annotated_data.csv')
X = pd.read_csv(path_to_group_data)

## remove unnecessary columns
borked_cols = ['Unnamed: 0','row_index']
for this_col in borked_cols:
    if this_col in X.columns:
        X = X.drop(labels=[this_col], axis=1)    

### render sketches using svg data (can be skipped if already rendered)

In [47]:
# collapse spline df to get only unique sketch svg strings
S = X.groupby('sketch_id')['sketch_svg_string'].unique().reset_index()

# extract svg string from additional level of nesting
S = S.assign(sketch_svg_string = S['sketch_svg_string'].apply(lambda x: x[0]))

# literal eval to convert string into list of actual svg
svg_string_list = [ast.literal_eval(i) for i in S.sketch_svg_string.values]
fname_list = S['sketch_id'].values ## extract list of image filenames


# add to dataframe
S = S.assign(svg_string_list = pd.Series(svg_string_list))
S = S.drop('sketch_svg_string',axis=1)

In [48]:
## render out svg & convert to png
reload(srh)
reallyRun = 1
if reallyRun:
    for this_fname,this_svg in zip(fname_list,svg_string_list):    
        srh.render_svg(this_svg,base_dir=sketch_dir,out_fname= '{}.svg'.format(this_fname))    
        print 'Rendering {}'.format(this_fname)
        clear_output(wait=True)
        
    ## get svg path list for rendered out svg
    svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))    
    
    ## convert all svg to png
    srh.svg_to_png(svg_paths,base_dir=sketch_dir)    

convert /Users/judithfan/semantic_parts/sketches/svg/9877-e1c91119-da46-40fa-82fe-17405ca56a3b_6.svg /Users/judithfan/semantic_parts/sketches/png/9877-e1c91119-da46-40fa-82fe-17405ca56a3b_6.png
